In [ ]:
!pip install --upgrade pip
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers pillow accelerate

In [ ]:
from huggingface_hub import login
login(token="<HF_token>")

In [ ]:
import torch

from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = "google/paligemma2-3b-pt-896"
processor = PaliGemmaProcessor.from_pretrained(model_id)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto").to(device)
model.eval()

In [ ]:
image_path = "caption-image-here.jpg"
image = Image.open(image_path).convert("RGB")

In [ ]:
prompt = ""
inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

input_length = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generated_ids = model.generate(**inputs, max_new_tokens=100, do_sample=False)
    caption_ids = generated_ids[0][input_length:]
    caption = processor.decode(caption_ids, skip_special_tokens=True)

print("CAPTION:", caption)